In [1]:
# Imports

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import random 

from adquire_functions import get_students_and_katas
from cleaning_funcions import clean_katas

In [2]:




def adquire(): 
    return get_students_and_katas()

def cleaning(df_students, df_katas): 
    return df_students, clean_katas(df_katas)


In [ ]:
# https://www.codewars.com/api/v1/users/Livia Canet/code-challenges/completed
# https://www.codewars.com/kata/

def get_completed_by_user(user, host='https://www.codewars.com/api/v1/users/', 
                          path='/code-challenges/completed'):
    url = host + user + path
    resp = requests.get(url)
    
    return resp.json()

def user_time_kata(user, slug, completed, katas): 
    res = '2090-01-01T00:00:00.000Z'# default
    for data in completed[user]: 
        if data['slug'] == slug: 
            res = data['completedAt']
            break
    return res

def create_df_checking(df_students, df_katas): 
    katas = df_katas.index
    students = df_students.index
    
    # Creamos un DataFrame: Students x Katas
    df_res = pd.DataFrame(index=df_students.index, columns=df_katas.index)
    
    # tarda un rato
    completed = {u: get_completed_by_user(u) for u in df_students.index}    
    completed_katas_dict = dict( [(k, v['data']) for k, v in completed.items()] )
       
    # cada celda es el tiempo que ese student(row) tardó en realizar la kata(column)
    for user in students: 
        for slug in katas: 
            df_res.loc[user, slug] = user_time_kata(user, slug, completed_katas_dict, katas)
    
    # convertimos las columnas a tipo Datetime
    for col in df_res.columns: 
        df_res[col] = pd.to_datetime(df_res[col],infer_datetime_format=True)
    # En este punto df_res son valores temporales
    
    # comprobamos si ocurren antes del límite
    for user in students: 
        df_res.loc[user] = df_res.loc[user] < df_katas.limit
    
    return df_res
    
    
# df = create_df_checking(df_students, df_katas)
# df.head()


In [ ]:
def save_results(df, path='results.csv'): 
    df.to_csv(path, index=True)
    
def choose_presenter_last_kata(df): 
    last_kata = df.columns[-1]
    completada = list(df[df[last_kata]].index)
    return random.choice(completada)

def display_presenter_last_kata(df): 
    student = choose_presenter_last_kata(df)
    last_kata = df.columns[-1]
    print('{} presentará la kata {}'.format(student, last_kata))

In [ ]:
   
    

def main(): 
    df_students, df_katas = adquire()
    df_students, df_katas = cleaning(df_students, df_katas)
    df = create_df_checking(df_students, df_katas)
    display(df)    
    save_results(df)
    display_presenter_last_kata(df)

if __name__ == '__main__':
    main()